References/Credits: 
* https://www.kaggle.com/ateplyuk/pytorch-efficientnet
* https://www.kaggle.com/chanhu/eye-efficientnet-pytorch-lb-0-777/data
* https://www.kaggle.com/abhinand05/blindness-detection-complete-pytorch-training
* https://www.kaggle.com/ratthachat/aptos-eye-preprocessing-in-diabetic-retinopathy

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import cv2
import seaborn as sns
from PIL import Image

from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
import torch.utils.data as utils

import torchvision
from torchvision import transforms


import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_dir = '../input/aptos2019-blindness-detection/'
train_dir = data_dir + '/train_images/'
test_dir = data_dir + '/test_images/'

In [ ]:
train_df = pd.read_csv(data_dir+"train.csv")
train_df.head()

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
IMG_SIZE = 224
seed_everything(13)

In [ ]:
N_FOLDS = 3
BATCH_SIZE = 32
N_EPOCHS = 7

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
folds = StratifiedKFold(n_splits=N_FOLDS, shuffle=True)

# Visualization

In [ ]:
counts = train_df['diagnosis'].value_counts()
class_list = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferate']
for i,x in enumerate(class_list):
    counts[x] = counts.pop(i)

plt.figure(figsize=(10,5))
sns.barplot(counts.index, counts.values, alpha=0.8, palette='bright')
plt.title('Distribution of Output Classes')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Target Classes', fontsize=12)
plt.show()

In [ ]:
fig = plt.figure(figsize=(25, 16))

for class_id in sorted(train_df.diagnosis.unique()):
    for i, (idx, row) in enumerate(train_df.loc[train_df['diagnosis'] == class_id].sample(5).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"{train_dir}{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

Now let's crop and see. I took the function crop_image_from_gray from this kernel: https://www.kaggle.com/ratthachat/aptos-eye-preprocessing-in-diabetic-retinopathy . Here at first if the image is 2 dimensional or 3 dimensional is checked. Regardles of the image being 2D or 3D, the black border is cropped using mask.any(0) and mask.any(1). This method is well explained here: https://codereview.stackexchange.com/questions/132914/crop-black-border-of-image-using-numpy

mask.any(1) or mask.any(axis=1) is applied the mask columnwise. Similarly mask.any(axis=0) applies the mask row-wise. These two boolean arrays are used to index the image and then crop the image (shown in the code below).

In [ ]:
fig = plt.figure(figsize=(5, 5))
for i in os.listdir(train_dir):
    image = cv2.imread(train_dir+i)
    gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    mask = gray_img>7

    
    plt.imshow(image[:,:,0][np.ix_(mask.any(1),mask.any(0))])
    break

In [ ]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
                #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [ ]:
class ImageData(Dataset):
    def __init__(self, df, data_dir, transform):
        super().__init__()
        self.df = df.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):       
        img_name,label = self.df[index]
        
        img_path = os.path.join(self.data_dir, img_name+'.png')
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = crop_image_from_gray(image)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , 30) ,-4 ,128)
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
data_transf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation((-180, 180)),
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.RandomVerticalFlip(p=0.5),
    #transforms.ColorJitter(brightness=2, contrast=2),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [ ]:
transforms_valid = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [ ]:
def train_one_fold(i_fold, model, loss_func, optimizer, train_loader, valid_loader):
    
    train_fold_results = []

    for epoch in range(N_EPOCHS):

        print('  Epoch {}/{}'.format(epoch + 1, N_EPOCHS))
        print('  ' + ('-' * 20))

        model.train()
        tr_loss = 0
        train_kappa = []
        optimizer.zero_grad()
        for ii, (data, target) in enumerate(train_loader):

            images = data.to(device, dtype = torch.float)
            labels = target.to(device, dtype = torch.float)
            
            labels = labels.view(-1,1)
            optimizer.zero_grad()
            
            
            with torch.set_grad_enabled(True):
                output = model(images)                
                loss = loss_func(output, labels)
                loss.backward()

                optimizer.step()
            
            tr_loss += loss.item()
            
            y_actual = labels.data.cpu().numpy()
            y_pred = output[:,-1].detach().cpu().numpy()
            kappa = cohen_kappa_score(y_actual, y_pred.round(), weights='quadratic')
            train_kappa.append(kappa)
        
        train_kappa_epoch = np.mean(train_kappa)
            

        # Validate
        model.eval()
        
        val_loss = 0
        val_preds = None
        val_labels = None
        val_kappa = []
        
        for ii, (data, target) in enumerate(valid_loader):


            images = data.to(device, dtype = torch.float)
            labels = target.to(device, dtype = torch.float)

            with torch.no_grad():
                outputs = model(images)

                loss = loss_func(outputs, labels.squeeze(-1))
                val_loss += loss.item()

            y_actual = labels.data.cpu().numpy()
            y_pred = outputs[:,-1].detach().cpu().numpy()
            kappa = cohen_kappa_score(y_actual, y_pred.round(), weights='quadratic')
            val_kappa.append(kappa)

           
        val_kappa_epoch = np.mean(val_kappa)
        train_loss = tr_loss/len(train_loader)
        valid_loss = val_loss/len(valid_loader)
        
        print('Fold: {}, Epoch: {}, Train Loss: {:.6f}, Valid Loss: {:.6f}, Train Kappa: {:.4f}, Valid Kappa: {:.4f}'.format(i_fold, epoch,train_loss, valid_loss, train_kappa_epoch, val_kappa_epoch))


In [ ]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

# Model
I used EfficientNet B3 for this dataset. I could not use Efficient Net B4 or higher as they kept failing due to memory shortage.   

In [ ]:
model = EfficientNet.from_name('efficientnet-b3')
model.load_state_dict(torch.load('../input/efficientnet-pytorch/efficientnet-b3-c8376fa2.pth'))
in_features = model._fc.in_features
model._fc = nn.Linear(in_features, 1)

In [ ]:
for i_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df.id_code, train_df.diagnosis)):
    print("Fold {}/{}".format(i_fold + 1, N_FOLDS))

    valid = train_df.iloc[valid_idx]
    valid.reset_index(drop=True, inplace=True)

    train = train_df.iloc[train_idx]
    train.reset_index(drop=True, inplace=True)    
    
    train_data = ImageData(df = train, data_dir = train_dir, transform = data_transf)
    dataset_valid = ImageData(df=valid, data_dir = train_dir, transform =transforms_valid)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
    valid_loader = DataLoader(dataset_valid, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)
    
    
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.00015, weight_decay=1e-5)
    loss_func = nn.MSELoss()
        
    train_one_fold(i_fold, model, loss_func, optimizer, train_loader, valid_loader)

torch.save(model.state_dict(), 'aptos_3_fold_weights.pth')